In [ ]:
# https://github.com/facebookresearch/esm/blob/main/README.md
import torch
import esm

import sys; sys.path.append("..")

In [ ]:
# Load an ESM-2 model - note that many different model sizes are available!

# ** 650M parameters **
# This model checkpoint is about 2.4 GB, and the embeddings can be retrieved from layer #33.
# Embeddings are 1280-dimensional.
# model, alphabet = esm.pretrained.esm2_t33_650M_UR50D() # from README
# model_repr_layer = 33

# ** 35M parameters **
# This model checkpoint is about 128 MB, and the embeddings are also from layer #12.
# Embeddings are 480-dimensional.
# model, alphabet = esm.pretrained.esm2_t12_35M_UR50D() # smallest 
# model_repr_layer = 12

# ** 8M parameters **
# This model checkpoint is about 29 MB, and the embeddings are also from layer #6.
# Embeddings are 320-dimensional.
model, alphabet = esm.pretrained.esm2_t6_8M_UR50D() # smallest 
model_repr_layer = 6

batch_converter = alphabet.get_batch_converter()
model.eval()  # disables dropout for deterministic results

In [ ]:
# Prepare data (first 2 sequences from ESMStructuralSplitDataset superfamily / 4)
# data = [
#     ("protein1", "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG"),
#     ("protein2", "KALTARQQEVFDLIRDHISQTGMPPTRAEIAQRLGFRSPNAAEEHLKALARKGVIEIVSGASRGIRLLQEE"),
#     ("protein2 with mask","KALTARQQEVFDLIRD<mask>ISQTGMPPTRAEIAQRLGFRSPNAAEEHLKALARKGVIEIVSGASRGIRLLQEE"),
#     ("protein3",  "K A <mask> I S Q"),
# ]

data = [
    ("protein1", "MKTVRQERLKSIVRILE"),
    ("protein2", "KALTARQQEVFDLIRDH"),
]

batch_converter = alphabet.get_batch_converter()

batch_labels, batch_strs, batch_tokens = batch_converter(data)
batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)

alphabet.all_toks
# batch_tokens

In [ ]:
# Extract per-residue representations (on CPU)
with torch.no_grad():
    results = model(batch_tokens, repr_layers=[model_repr_layer], return_contacts=True)
token_representations = results["representations"][model_repr_layer]
token_representations[0].shape

token_representations[:, 0, :10]

In [ ]:
# Generate per-sequence representations via averaging
# NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
sequence_representations = []
for i, tokens_len in enumerate(batch_lens):
    sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))

# NOTE(milo): This model produces 1280-dimensional embeddings.
sequence_representations[0].shape

In [ ]:
# Look at the unsupervised self-attention map contact predictions
import matplotlib.pyplot as plt
for (_, seq), tokens_len, attention_contacts in zip(data, batch_lens, results["contacts"]):
    plt.matshow(attention_contacts[: tokens_len, : tokens_len])
    plt.title(seq)
    plt.show()

In [ ]:
import gvpgnn.embeddings as embeddings

In [ ]:
model_factory = embeddings.esm2_model_dictionary["esm2_t6_8M_UR50D"]
layer = embeddings.esm2_embedding_layer["esm2_t6_8M_UR50D"]

model, alphabet = model_factory()

seq = "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG<unk>"
out = embeddings.extract_embedding_single(model, alphabet, layer, seq)

# out.shape
print(out.shape)
print(len(seq))

out